In [49]:
%env FORCE_CMAKE=1
%env CMAKE_ARGS=-DLLAMA_CUDA=on

!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121
!pip install langchain-community langchain_groq
!pip install -U ddgs

env: FORCE_CMAKE=1
env: CMAKE_ARGS=-DLLAMA_CUDA=on
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 14.0 MB/s eta 0:00:00


In [50]:
import os
from google.colab import userdata

from langchain_community.llms import LlamaCpp
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.agent_toolkits.load_tools import load_tools

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import Tool

from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_react_agent

from langchain_groq import ChatGroq

from huggingface_hub import hf_hub_download

In [3]:
model_path = hf_hub_download(
    repo_id="MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf",
    local_dir="."
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
llm = LlamaCpp(
    model_path="/content/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=1024,
    temperature=0,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (1024) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [5]:
template_title = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Create a short, creative title for a story about {summary}. Only return the title.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

title_prompt = ChatPromptTemplate.from_template(template_title)

template_character = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

character_prompt = ChatPromptTemplate.from_template(template_character)

template_story = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

story_prompt = ChatPromptTemplate.from_template(template_story)

In [6]:
title_step = title_prompt | llm | StrOutputParser()

character_step = (
    {"summary": lambda x: x["summary"], "title": lambda x: x["title"]}
    | character_prompt
    | llm
    | StrOutputParser()
)

story_step = story_prompt | llm | StrOutputParser()

chain = (
    RunnableParallel(summary=RunnablePassthrough())
    | RunnablePassthrough.assign(title=title_step)
    | RunnablePassthrough.assign(character=character_step)
    | RunnablePassthrough.assign(story=story_step)
)

out = chain.invoke("A boy who fought Kung-Fu")

for key in list(out.keys()):
    print(key)
    print(out[key])

summary
A boy who fought Kung-Fu
title


"The Dragon's Fury: A Kung-Fu Legend"
character


The main character, a young boy named Ling, is a skilled and fierce Kung-Fu master who wields the power of the dragon. With his unwavering determination and unrelenting fury, Ling is destined to become a legendary hero in the world of Kung-Fu.
story


The Dragon's Fury: A Kung-Fu Legend. Ling, a young boy with an unyielding determination and unrelenting fury, is a skilled and fierce Kung-Fu master who wields the power of the dragon. With his unwavering dedication to justice and his unrelenting passion for Kung-Fu, Ling is destined to become a legendary hero in the world of Kung-Fu.


Simple agent

In [51]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

llm_agent = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [65]:
search = DuckDuckGoSearchResults()

search_tool = Tool(
    name="duckduckgo",
    description="Use this for current events or technical facts you don't know.",
    func=search.run,
)

tools = load_tools(["llm-math"], llm=llm_agent)
tools.append(search_tool)

In [66]:
agent_prompt = hub.pull("hwchase17/react")

In [67]:
agent = create_react_agent(llm_agent, tools, agent_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [77]:
agent_executor.invoke({"input": "What is the current price of a 14-inch MacBook Pro in USD? Return me the price and a single updated website with this information."})



> Entering new AgentExecutor chain...
Thought: To find the current price of a 14-inch MacBook Pro in USD, I need to search for the latest information on the web. The price may vary depending on the region, storage, and other specifications, so I'll need to find a reliable source that provides up-to-date pricing information.

Action: duckduckgo
Action Input: "14-inch MacBook Pro price USD"snippet: 2 weeks ago - ... This particular configuration has an upgraded M4 Pro chip with a 14-core CPU and 20-core GPU. It also has 1TB of storage, which is a step up from the standard 512GB in the M4 Pro line. The $2,099 price is the lowest available in the past ..., title: Save $300 on 14-inch M4 Pro MacBook Pro with 1TB SSD, link: https://appleinsider.com/articles/26/01/12/price-war-save-300-on-apples-1tb-14-inch-macbook-pro-with-20-core-gpu, snippet: October 15, 2025 - Find the top deals on 14-inch MacBook Pro laptops. Compare prices across trusted retailers to secure the best savings on your ne

{'input': 'What is the current price of a 14-inch MacBook Pro in USD? Return me the price and a single updated website with this information.',
 'output': 'The current price of a 14-inch MacBook Pro in USD is $1,399, and you can find the latest pricing information on websites such as https://appleinsider.com/articles/26/01/05/apples-m5-macbook-pro-drops-to-1399-200-off-as-ces-2026-kicks-off.'}

In [78]:
print(llm_agent.invoke("What is the current price of a 14-inch MacBook Pro in USD? Return me the price and a single updated website with this information."))

content="As of my knowledge cutoff in 2023, the prices for a 14-inch MacBook Pro may vary depending on the configuration. However, I can provide you with the base model prices and a website where you can find the most up-to-date information.\n\nThe base model prices for a 14-inch MacBook Pro (2023) are:\n- 14-inch MacBook Pro with M2 chip, 8GB RAM, and 512GB SSD: around $1,999\n- 14-inch MacBook Pro with M2 Pro chip, 16GB RAM, and 512GB SSD: around $2,299\n- 14-inch MacBook Pro with M2 Pro chip, 16GB RAM, and 1TB SSD: around $2,499\n- 14-inch MacBook Pro with M2 Max chip, 32GB RAM, and 1TB SSD: around $3,099\n\nFor the most up-to-date pricing information, you can visit Apple's official website: https://www.apple.com/macbook-pro-14/\n\nPlease note that prices may have changed since my knowledge cutoff, and I recommend checking the website for the latest information." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 235, 'prompt_tokens': 63, 'total_tokens': 29